In [1]:
from sklearn.linear_model import Lasso, LassoCV, RidgeCV
import numpy as np

class HAL:
    def __init__(self, *args, **kwargs):
        self.lasso = LassoCV(*args, **kwargs)

    def _basis_products(self, arr, index=0, current=None, result=None):
        if result is None:
            result = []
        if current is None:
            current = np.ones_like(arr[0], dtype=bool)

        if index == len(arr):
            result.append(current)
        else:
            self._basis_products(arr, index + 1, current & arr[index], result)
            self._basis_products(arr, index + 1, current, result)

        return result

    def _bases(self, X):
        one_way_bases = np.stack([
            np.less.outer(self.knots[:,j], X[:,j])
            for j in range(self.knots.shape[1])
        ])
        bases = self._basis_products(one_way_bases)
        return np.concatenate(bases[:-1]).T

    def fit(self, X, Y):
        self.knots = X
        self.lasso.fit(self._bases(X), Y) # (HᵀH + λI)^{-1}HᵀY = β

    def predict(self, X):
        return self.lasso.predict(self._bases(X)) # Hβ


In [2]:
import numpy as np

class dgp:
    @classmethod
    def f(cls, X):
        return -0.5*X[:,0] + (X[:,1] * X[:,0]**2) / 2.75 + X[:,1]

    @classmethod
    def gen(cls, n):
        X = np.column_stack((
            np.random.uniform(-4, 4, n),
            np.random.binomial(1, 0.5, n),
        ))
        Y = cls.f(X) + np.random.normal(0, 1, n)
        return X, Y

In [3]:
X,Y = dgp.gen(100)
X_test, Y_test = dgp.gen(10000)

In [4]:
model = HAL()
model.fit(X,Y)
model.lasso.alphas_

array([0.81515355, 0.76021493, 0.70897899, 0.66119617, 0.61663377,
       0.57507471, 0.5363166 , 0.50017066, 0.46646083, 0.43502293,
       0.40570384, 0.37836076, 0.35286051, 0.32907889, 0.30690008,
       0.28621604, 0.26692604, 0.24893612, 0.23215865, 0.21651194,
       0.20191976, 0.18831104, 0.17561951, 0.16378334, 0.15274489,
       0.1424504 , 0.13284972, 0.12389609, 0.11554591, 0.1077585 ,
       0.10049594, 0.09372285, 0.08740624, 0.08151535, 0.07602149,
       0.0708979 , 0.06611962, 0.06166338, 0.05750747, 0.05363166,
       0.05001707, 0.04664608, 0.04350229, 0.04057038, 0.03783608,
       0.03528605, 0.03290789, 0.03069001, 0.0286216 , 0.0266926 ,
       0.02489361, 0.02321587, 0.02165119, 0.02019198, 0.0188311 ,
       0.01756195, 0.01637833, 0.01527449, 0.01424504, 0.01328497,
       0.01238961, 0.01155459, 0.01077585, 0.01004959, 0.00937228,
       0.00874062, 0.00815154, 0.00760215, 0.00708979, 0.00661196,
       0.00616634, 0.00575075, 0.00536317, 0.00500171, 0.00466

In [5]:
model.lasso.alpha_

0.014245039754648126

In [6]:
model = HAL()
model.fit(X,Y)

In [7]:
Ŷ = model.predict(X_test)

In [8]:
np.mean((Y_test - Ŷ)**2)

1.354553701520526

In [9]:
model = HAL()
model.fit(X,Y)

In [10]:
Ŷ = model.predict(X_test)

In [11]:
np.mean((Y_test - Ŷ)**2)

1.354553701520526